In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
! pip install torch==2.1.0  torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
#! pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
! pip install torch_geometric
! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
#! pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
#! pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
#! pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
! pip install sentence-transformers
! pip install torcheval
! pip install matplotlib
! pip install pandas
! pip install tensorboard
! pip install weaviate-client

! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download en_core_web_sm

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4fdce78aa74a177bf250fb4ecc394357eac00e6337f640b1130a180389ac513c
  Stored in direc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2023-12-09 14:30:03.678295: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 14:30:03.678362: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempt

In [2]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Google Colab")
else:
    print("Not running in Google Colab")


Running in Google Colab


In [3]:
if IN_COLAB:
  import os
  import numpy as np
  from pathlib import Path
  from tqdm.auto import tqdm
  from google.colab import drive
  import matplotlib.pyplot as plt
  drive.mount('/content/drive',force_remount=True)
  DRIVE_FOLDER = Path('/content/drive/MyDrive/Data/')
  DRIVE_GNN_PROGESS_FOLDER = DRIVE_FOLDER/'GNNPaper_Search'
  ROOT_FOLDER = DRIVE_GNN_PROGESS_FOLDER

Mounted at /content/drive


In [5]:
ROOT_FOLDER ='src/'
import shutil

if IN_COLAB and not 'unzipped' in globals():
  #!unzip /content/drive/MyDrive/Data/HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt
  source_path = "/content/drive/MyDrive/Data/HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt"
  destination_path = "/content/dataset.pt"

  #unzipped = True
  shutil.copy(source_path, destination_path)

In [7]:
from torch_geometric.nn import HGTConv, Linear
import torch

class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers, node_types, data_metadata):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data_metadata,
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict = {
            node_type: self.lin_dict[node_type](x).relu_()
            for node_type, x in x_dict.items()
        }

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return x_dict


OSError: ignored

In [ ]:
from torch_geometric.nn.kge import TransE

import math

import torch
import torch.nn.functional as F
from torch import Tensor

from torch_geometric.nn.kge import KGEModel

# adapted and taken from https://github.com/pyg-team/pytorch_geometric/blob/master/torch_geometric/nn/kge/transe.py

class TransE(KGEModel):
    r"""The TransE model from the `"Translating Embeddings for Modeling
    Multi-Relational Data" <https://proceedings.neurips.cc/paper/2013/file/
    1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf>`_ paper.

    :class:`TransE` models relations as a translation from head to tail
    entities such that

    .. math::
        \mathbf{e}_h + \mathbf{e}_r \approx \mathbf{e}_t,

    resulting in the scoring function:

    .. math::
        d(h, r, t) = - {\| \mathbf{e}_h + \mathbf{e}_r - \mathbf{e}_t \|}_p

    .. note::

        For an example of using the :class:`TransE` model, see
        `examples/kge_fb15k_237.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        kge_fb15k_237.py>`_.

    Args:
        num_nodes (int): The number of nodes/entities in the graph.
        num_relations (int): The number of relations in the graph.
        hidden_channels (int): The hidden embedding size.
        margin (int, optional): The margin of the ranking loss.
            (default: :obj:`1.0`)
        p_norm (int, optional): The order embedding and distance normalization.
            (default: :obj:`1.0`)
        sparse (bool, optional): If set to :obj:`True`, gradients w.r.t. to the
            embedding matrices will be sparse. (default: :obj:`False`)
    """
    def __init__(
        self,
        num_nodes: int,
        num_relations: int,
        hidden_channels: int,
        margin: float = 1.0,
        p_norm: float = 1.0,
        sparse: bool = False,
    ):
        super().__init__(num_nodes, num_relations, hidden_channels, sparse)

        self.p_norm = p_norm
        self.margin = margin

        self.reset_parameters()

    def reset_parameters(self):
        bound = 6. / math.sqrt(self.hidden_channels)
        torch.nn.init.uniform_(self.node_emb.weight, -bound, bound)
        torch.nn.init.uniform_(self.rel_emb.weight, -bound, bound)
        F.normalize(self.rel_emb.weight.data, p=self.p_norm, dim=-1,
                    out=self.rel_emb.weight.data)

    def forward(
        self,
        head_embeddings: Tensor,
        rel_type,
        tail_embeddings: Tensor,
    ) -> Tensor:
        #head = self.node_emb(head_index)
        rel = self.rel_emb(rel_type)  # Amos: only learn the relation embeddings, others are learned with GNN
        #tail = self.node_emb(tail_index)

        head = F.normalize(head_embeddings, p=self.p_norm, dim=-1)
        tail = F.normalize(tail_embeddings, p=self.p_norm, dim=-1)

        # Calculate *negative* TransE norm:
        return -((head + rel) - tail).norm(p=self.p_norm, dim=-1)


    def get_embedding(self,
                      embedding,
                      rel_type,
                        have_head_or_tail
                      ):
        rel = self.rel_emb(rel_type)
        embedding = F.normalize(embedding, p=self.p_norm, dim=-1)
        if have_head_or_tail == 'head':
            return embedding + rel
        else:
            return embedding - rel


    def loss(
        self,
        head_embeddings: Tensor,
        rel_type: Tensor,
        tail_embeddings: Tensor,
        labels: Tensor, # labels 0 or 1
    ) -> Tensor:
        pos_mask = labels == 1
        neg_mask = labels == 0

        pos_score = self(head_embeddings[pos_mask], rel_type, tail_embeddings[pos_mask])
        neg_score = self(head_embeddings[neg_mask], rel_type, tail_embeddings[neg_mask])

        return F.margin_ranking_loss(
            pos_score,
            neg_score,
            target=torch.ones_like(pos_score), # 1 for similarity, -1 for dissimilarity
            margin=self.margin,
        )

In [ ]:
# GraphSampler
# COMMAND ----------
# Sampling
# 1. Sample using HGT Sampler as outlined in the paper, using pyg implementations
# 2. The sampling is adapted to link prediction, by first sampling random supervision edges of which the nodes create the supervision nodes
# a. Dataset is divided across multiple dimensions:
#   a.1. Split into Train, Val, Test split (96, 2, 2)
#   a.2. Training only: Edges are split into those which are used solely for message passing and those solely used for supervision (80, 20).
#        Because an expressive model (HGT) is used, this prevents the model from memorizing supervision edges by their appearance as message passing edges
#   a.3. This means Training consists of 96%*80% Message passing Edges, 96%*20% supervision edges, Val contains 2% Supervision Edges, Test contains 2% supervison Edges
#   a.4. Validation and Test edges use the Training Message passing Edges as well.
# b. For mini-batch sampling in the training phase, first x random edges are sampled as supervision edges.
#    For the nodes of these supervision edges, we apply batch-wise HGT Sampling. Due to implementation limitations, for each supervision entity type, the hgt sampling is separate.
#    This limitation does not apply for sampled neighbor entity types
# during sampling, also the reverse edge of the supervision edge is removed to avoid data leakage


# HGT Sampler (See Paper for further reference)
# The probablity of a neighbor node s to be sampled depends on the normalized degree of all its edge types connecting it to all source nodes
# If neighbor node s is connected to a and b by edge type r, and a has 2 neighbors through edge type r and b has 1 neighbor (node s) through edge type r,
# then the sampling probablity of s is (1/2+1)**2 / 2**2, if it were connected through other edge types to the nodes as well, those degrees would be added to the numerator and denominator.
# Nodes are sampled without replacement.
# This sampling strategy creates more dense mini-batches, because neighbor nodes which are connected to multiple source nodes and by multiple relationship types are sampled more frequently.
# Therefore, training is sped up since less node representations have to be computed. Furthermore, as stated in the paper, the sampling method allows to sample a
# similar number of neighbors for each edge type, because high-count edge types and low-count edge types are weighted equally. For each neighbor node type T, a fixed number n of nodes is sampled.





import os
import torch
from torch_geometric.data import HeteroData
from tqdm.auto import tqdm
import numpy as np
def add_reverse_edge_original_attributes_and_label_inplace(original_edge, reverse_edge):
    # add edge label and index and edge attr to reverse edge
    if 'edge_attr' in original_edge:
        reverse_edge['edge_attr'] = original_edge['edge_attr']
    reverse_edge['edge_label'] = original_edge['edge_label']
    reverse_edge['edge_label_index'] = original_edge['edge_label_index'].index_select(0, torch.LongTensor([1, 0]))
    for key in original_edge.keys():
        if key not in ['edge_index', 'edge_attr', 'edge_label', 'edge_label_index']:
            reverse_edge[key] = original_edge[key]

    return reverse_edge

def get_datasets(get_edge_attr=False, filename=None, filter_top_k=False, top_k=50, remove_text_attr=True):
    if filename is None:
        filename = 'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'
    size = os.path.getsize(filename)
    print('size of dataset on disk: ', size/1e9, 'gb')

    if os.path.exists(filename):
        data = HeteroData.from_dict(torch.load(filename))
        print('loading saved heterodata object')



    def top_k_mask(scores, indices, top_k ):
        # Make sure we are using the GPU
        scores = scores.cuda()
        indices = indices.cuda()

        # Create an empty mask with the same shape as scores
        mask = torch.zeros_like(scores, dtype=torch.bool)
        # Get the unique indices and their counts
        unique_indices, counts = torch.unique(indices, return_counts=True)

        # Indices where count > top_k
        large_indices = unique_indices[counts > top_k]

        # Set mask for indices where count <= top_k
        mask[~torch.isin(indices,large_indices)] = True
        # For indices where count > 50, we only keep top 50 scores
        for idx in tqdm(large_indices):
            idx_mask = (indices == idx)
            values, idxs = scores[idx_mask].topk(top_k)
            a = mask[idx_mask]
            a[idxs] = True
            mask[idx_mask] = a

        return mask.cpu()



    if filter_top_k:
        print('for skill job edges keep top k edges per job, k is ',top_k)
        e = ('skills', 'job_skill', 'jobs')
        rev_e = (e[2],'rev_'+e[1],e[0])
        cache_dir = 'cache'
        if not os.path.exists(cache_dir):
            os.makedirs(cache_dir)

        mask_path = os.path.join(cache_dir, f'mask{top_k}.pt')

        if os.path.isfile(mask_path):
            mask = torch.load(mask_path)
        else:
            mask = top_k_mask(data[e].edge_attr.squeeze(1), data[e].edge_index[1,:], top_k)
            torch.save(mask, mask_path)

        data[e].edge_attr = data[e].edge_attr[mask]
        data[rev_e].edge_attr = data[rev_e].edge_attr[mask]
        data[e].edge_index = data[e].edge_index[:,mask]
        data[rev_e].edge_index = data[rev_e].edge_index[:,mask]
        print('keep',torch.sum(mask), 'of total',mask.shape[0])


    from torch_geometric import seed_everything
    import torch_geometric.transforms as T
    from torch_geometric.utils import sort_edge_index



    edge_types = []
    rev_edge_types = []
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            rev_edge_types.append(edge_type)
        else:
            edge_types.append(edge_type)

    transform = T.RandomLinkSplit(
        is_undirected=True,
        edge_types=edge_types,
        rev_edge_types=rev_edge_types,
        num_val=0.02,
        num_test=0.05,
        add_negative_train_samples=False, # only adds neg samples for val and test, neg train are added by LinkNeighborLoader. This means for each train batch, negs. are different, for val and train they stay the same
        neg_sampling_ratio=1.0,
        disjoint_train_ratio=0.3, #  training edges are shared for message passing and supervision
        )

    seed_everything(14)
    # sort by col to speed up sampling later (we can sepcify is_sorted=True in link neighbor loader)
    # we actually dont use the sort, because it seems to mess up things, but have not checked if everything works without sorting, so we leave it here
    def sort_edges(data):
        for edge_type in data.edge_types:
            if 'edge_attr' in data[edge_type].keys():
                data[edge_type].edge_index, data[edge_type].edge_attr = sort_edge_index(data[edge_type].edge_index, data[edge_type].edge_attr, sort_by_row=False)
            else:
                data[edge_type].edge_index = sort_edge_index(data[edge_type].edge_index, sort_by_row=False)
        return data


    def preprocess(data):
        if not get_edge_attr:
            # delete edge_attr of every edge type
            for edge_type in data.edge_types:
                del data[edge_type].edge_attr

        # delete all keys for every node type except 'x' (e.g. description and title)
        for node_type in data.node_types:
            keys = list(data[node_type].keys())
            for key in keys:
                if key != 'x':
                    del data[node_type][key]
        return data


    # change all types to float32 and normalize the triangle columns
    for node_type in data.node_types:
        for i in range(data[node_type].x.shape[1]):
            if data[node_type].x[:,i].max()>5:
                #normalize
                print('normalizing column ', i, ' of node type ', node_type)
                data[node_type].x[:,i] = data[node_type].x[:,i]/data[node_type].x[:,i].max()

        data[node_type].x = data[node_type].x.to(torch.float32)



    train_data, val_data, test_data = transform(data)
    #train_data = sort_edges(train_data)
    #val_data = sort_edges(val_data)
    #test_data = sort_edges(test_data)
    if remove_text_attr:
        train_data = preprocess(train_data)
        val_data = preprocess(val_data)
        test_data = preprocess(test_data)

    return train_data, val_data, test_data




from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.loader import HGTLoader
from torch_geometric.sampler import NegativeSampling

def get_hgt_linkloader(data, target_edge, batch_size, is_training, sampling_mode, neg_ratio, num_neighbors_hgtloader, num_workers, prefetch_factor, pin_memory):
    # first sample some edges in linkNeighborLoader
    # use the nodes of the sampled edges to sample from hgt loader


    num_neighbors_linkloader = [0]
    #for edge_type in data.edge_types:
    #    num_neighbors_linkloader[edge_type] = [0,0]

    negative_sampling = NegativeSampling(
        mode=sampling_mode, # binary or triplet
        amount=neg_ratio  # ratio, like Graphsage # 10
        #weight=  # "Probabilities" of nodes to be sampled: Node degree follows power law distribution
        )

    if sampling_mode == 'triplet':
        data[target_edge].edge_label = None


    linkNeighborLoader = LinkNeighborLoader(
            data,
            num_neighbors=num_neighbors_linkloader,
            edge_label_index=(target_edge, data[target_edge].edge_label_index), # if (edge, None), None means all edges are considered

            neg_sampling=negative_sampling, # adds negative samples
            batch_size=batch_size,
            shuffle=is_training, #is_training
            subgraph_type='directional', # contains only sampled edges
            #drop_last=True,
            num_workers=num_workers,
            #disjoint=True # sampled seed node creates its own, disjoint from the rest, subgraph, will add "batch vector" to loader output

            #num_workers=2,
            #prefetch_factor=2
            is_sorted = False,
            pin_memory=pin_memory,
            prefetch_factor=prefetch_factor,
    )


    def get_hgt(data, input_nodetype, input_mask):
        return next(iter(HGTLoader(
                data,
                # Sample 512 nodes per type and per iteration for 4 iterations
                num_samples=num_neighbors_hgtloader,
                batch_size=input_mask.shape[0],
                input_nodes=(input_nodetype, input_mask),
                num_workers=num_workers,
                pin_memory=pin_memory,
                prefetch_factor=prefetch_factor,
            )))


    def add_self_loops(data):
        for node_type in data.node_types:
            data[node_type, 'self_loop', node_type].edge_index = torch.arange(data[node_type].num_nodes).repeat(2,1)
        return data


    def get_hgt_with_selfloops(loader):


        for batch in loader:
            if sampling_mode=='triplet':
                # original edge_label_index from the whole data object
                unmapped_batchids = torch.cat((batch[target_edge[0]].src_index,batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index.flatten()))
                original_node_ids = batch[target_edge[0]].n_id[unmapped_batchids]
                original_edge_label_nodes = torch.LongTensor(original_node_ids.unique())

                #remapping or sorting is not needed, since nodes are sorted, also in the htg batch, the edges will be the same
                src = batch[target_edge[0]].src_index.unsqueeze(0)
                src_total = src
                for i in range(neg_ratio):
                    src_total = torch.cat((src_total,src), dim=1)
                dst = torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index.flatten()),dim=0).unsqueeze(0)

                local_edge_label_index = torch.cat((src_total, dst),dim=0)
                edge_label = torch.cat((torch.ones(batch[target_edge[2]].dst_pos_index.shape[0]), torch.zeros(batch[target_edge[2]].dst_neg_index.flatten().shape[0])))

            elif sampling_mode=='binary':
                unmapped_batchids = batch[target_edge].edge_label_index.flatten()
                original_node_ids = batch[target_edge[0]].n_id[unmapped_batchids]
                original_edge_label_nodes = torch.LongTensor(original_node_ids.unique())

            else:
                raise Exception('binary or triplet sampling mode')


            hgt_batch = get_hgt(data, target_edge[0], original_edge_label_nodes) # 0,1,3,4,5,6,7,8,9,

            if sampling_mode=='triplet':

                # return message passing edges, and supervision edges/labels, ignore labels/label_indices in the message passing edges
                yield add_self_loops(hgt_batch), local_edge_label_index, edge_label, batch[target_edge].input_id, original_node_ids
            else: # sampling_mode=='binary':
                # return message passing edges, and supervision edges/labels, ignore labels/label_indices in the message passing edges, as well as original edge indices
                yield add_self_loops(hgt_batch), batch[target_edge].edge_label_index, batch[target_edge].edge_label, batch[target_edge].input_id, original_node_ids

    def get_hgt_2types_with_selfloops(loader):
        for batch in loader:
            if sampling_mode=='triplet':
                original_src_nodes = batch[target_edge[0]].n_id[batch[target_edge[0]].src_index]
                original_edge_label_nodes_class1 = torch.LongTensor(original_src_nodes.unique())

                original_dst_nodes = batch[target_edge[2]].n_id[torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index.flatten()))]
                original_edge_label_nodes_class2 = torch.LongTensor(original_dst_nodes.unique())


                src = batch[target_edge[0]].src_index.unsqueeze(0)
                src_total = src
                for i in range(neg_ratio):
                    src_total = torch.cat((src_total,src), dim=1)

                dst = torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index.flatten()),dim=0).unsqueeze(0)

                local_edge_label_index = torch.cat((src_total, dst),dim=0)
                edge_label = torch.cat((torch.ones(batch[target_edge[2]].dst_pos_index.shape[0]), torch.zeros(batch[target_edge[2]].dst_neg_index.flatten().shape[0])))

            elif sampling_mode=='binary':
                original_src_nodes = batch[target_edge[0]].n_id[batch[target_edge].edge_label_index[0,:]]
                original_edge_label_nodes_class1 = original_src_nodes.unique()
                original_dst_nodes = batch[target_edge[2]].n_id[batch[target_edge].edge_label_index[1,:]]
                original_edge_label_nodes_class2 = original_dst_nodes.unique()

            else:
                raise Exception('binary or triplet sampling mode')

            # batch the start and end supervision nodes separately
            hgt_batch1 = get_hgt(data, target_edge[0], original_edge_label_nodes_class1)
            hgt_batch2 = get_hgt(data, target_edge[2], original_edge_label_nodes_class2)


            # ** We dont need to remove any edges ** since the supervision edges wont be sampled by hgt
            if sampling_mode=='triplet':
                yield add_self_loops(hgt_batch1), add_self_loops(hgt_batch2), local_edge_label_index, edge_label, batch[target_edge].input_id, original_src_nodes, original_dst_nodes
            else: # sampling_mode=='binary':
                # we can access the corresponding nodes of edge_label_index[0,:] in hgt_batch1[target_edge[0]], those of [1,:] in hgt_batch2...
                yield add_self_loops(hgt_batch1), add_self_loops(hgt_batch2), batch[target_edge].edge_label_index, batch[target_edge].edge_label, batch[target_edge].input_id, original_src_nodes, original_dst_nodes


    if target_edge[0] == target_edge[2]:
        # same edge type, only need to sample once
        return get_hgt_with_selfloops(linkNeighborLoader)
    else:
        return get_hgt_2types_with_selfloops(linkNeighborLoader)


import random

def get_minibatch_count(data, batch_size):
    batches = []
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batches.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])

    return len(batches)

def get_single_minibatch_count(data, batch_size, edge_type):
    return (data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size

def uniform_hgt_sampler(data, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory):

    # return batches from all edgetypes with each "edge" being drawn uniformly at random (but we translate the probabilities to batches), last batches of each edge type may be smaller than batch_size
    batches = []
    loaders = {}
    # only the non-reverse edge types for now
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batches.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        loaders[edge_type]=get_hgt_linkloader(data, edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)

    random.seed(14)
    random.shuffle(batches)
    # set a random random seed again (may affect creating the loaders later for a second epoch)
    random.seed()

    print('total batches:', len(batches))

    for target_edge_type in batches:
        if target_edge_type[0] == target_edge_type[2]:
            same_nodetype = True
        else:
            same_nodetype = False
        try:
            minibatch = next(loaders[target_edge_type])
        except StopIteration: # "reinit" iterator
            loaders[target_edge_type] = get_hgt_linkloader(data, target_edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)#iter(loaders[target_edge_type])
            minibatch = next(loaders[target_edge_type])
            pass

        yield same_nodetype, target_edge_type, minibatch

def sampler_for_init(data, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory):
    batchcount=[]
    batches=[]
    loaders = {}
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batchcount.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        batches.append(edge_type)
        loaders[edge_type]=get_hgt_linkloader(data, edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)

    print('total batches:', len(batchcount))


    # set a random random seed again (may affect creating the loaders later for a second epoch)
    random.seed()

    for target_edge_type in batches:
        if target_edge_type[0] == target_edge_type[2]:
            same_nodetype = True
        else:
            same_nodetype = False
        try:
            minibatch = next(loaders[target_edge_type])
        except StopIteration: # "reinit" iterator
            loaders[target_edge_type] = get_hgt_linkloader(data, target_edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)#iter(loaders[target_edge_type])
            minibatch = next(loaders[target_edge_type])
            pass

        yield same_nodetype, target_edge_type, minibatch

def equal_edgeweight_hgt_sampler(data, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory):
    batchcount=[]
    batches=[]
    loaders = {}
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batchcount.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        batches.append(edge_type)
        loaders[edge_type]=get_hgt_linkloader(data, edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)

    print('total batches:', len(batchcount))

    batches = random.choices(batches, k=len(batchcount))
    # set a random random seed again (may affect creating the loaders later for a second epoch)
    random.seed()



    for target_edge_type in batches:
        if target_edge_type[0] == target_edge_type[2]:
            same_nodetype = True
        else:
            same_nodetype = False
        try:
            minibatch = next(loaders[target_edge_type])
        except StopIteration: # "reinit" iterator
            loaders[target_edge_type] = get_hgt_linkloader(data, target_edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)#iter(loaders[target_edge_type])
            minibatch = next(loaders[target_edge_type])
            pass

        yield same_nodetype, target_edge_type, minibatch

In [ ]:
#from graph_sampler import get_datasets, equal_edgeweight_hgt_sampler, get_minibatch_count, add_reverse_edge_original_attributes_and_label_inplace, get_hgt_linkloader, get_single_minibatch_count, sampler_for_init

train_data, val_data, test_data = get_datasets(get_edge_attr=False, filename=destination_path, filter_top_k=True, top_k=50)


/home/amos/mambaforge/envs/pyg_torch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


In [ ]:
import torch
from models.TransE import TransE
from models.DistMult import DistMult
from models.HGT import HGT
import torch_geometric
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Model(torch.nn.Module):
    def __init__(self, gnn : torch.nn.Module, head :  torch.nn.Module, node_types, edge_types, ggn_output_dim, pnorm=1):
        super().__init__()
        # edge_type onehot lookup table with keys
        # node_type onehot lookup table with keys
        self.node_type_embedding = torch.nn.Embedding(len(node_types), ggn_output_dim) # hidden channels should be the output dim of gnn

        self.edge_types = edge_types
        for edge_type in edge_types:
            if edge_type[1].startswith('rev_'):
                self.edge_types.remove(edge_type)

        # create edge to int mapping
        self.edgeindex_lookup = {edge_type:torch.tensor(i)  for i, edge_type in enumerate(edge_types)}

        # hidden channels should be the output dim of gnn
        if head=='TransE':
            self.head = TransE(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm, margin=0.5)  # KGE head with loss function
        elif head=='DistMult':
            self.head = DistMult(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm, margin=0.5)  # KGE head with loss function
        else:
            raise NotImplementedError

        self.gnn = gnn



    def forward(self, hetero_data1, target_edge_type, edge_label_index, edge_label, hetero_data2=None, get_head_fn='loss'):

        if hetero_data2 is not None:
            assert target_edge_type[0] != target_edge_type[2], 'when passing two data objects, the edge type has to contain two different node types'
            head_embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = self.gnn(hetero_data2.x_dict, hetero_data2.edge_index_dict)[target_edge_type[2]][edge_label_index[1,:]]
        else:
            assert target_edge_type[0] == target_edge_type[2], 'when passing one data object, the edge type has to contain the same node types'


            embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)
            head_embeddings = embeddings[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = embeddings[target_edge_type[2]][edge_label_index[1,:]]


        edgeindex = self.edgeindex_lookup[target_edge_type]
        if get_head_fn=='loss':
            loss = self.head.loss(head_embeddings, edgeindex.to(device), tail_embeddings, edge_label)
            return loss
        elif get_head_fn=='forward':
            return self.head.forward(head_embeddings, edgeindex.to(device), tail_embeddings)


metadata = train_data.metadata()
# add selfloops
for node_type in train_data.node_types:
    metadata[1].append((node_type, 'self_loop', node_type))

out_channels = 256
hidden_channels = 256
num_heads = 8
num_layers = 2
pnorm = 2
head = 'TransE'
gnn = HGT(hidden_channels=out_channels, out_channels=out_channels, num_heads=num_heads, num_layers=num_layers, node_types=train_data.node_types, data_metadata=metadata)

model = Model(gnn, head=head, node_types=metadata[0], edge_types=metadata[1], ggn_output_dim=out_channels, pnorm=pnorm)
#torch_geometric.compile(model, dynamic=True)
model.to(device)


Model(
  (node_type_embedding): Embedding(6, 256)
  (head): TransE(6, num_relations=22, hidden_channels=256)
  (gnn): HGT(
    (lin_dict): ModuleDict(
      (courses_and_programs): Linear(-1, 256, bias=True)
      (qualifications): Linear(-1, 256, bias=True)
      (skills): Linear(-1, 256, bias=True)
      (people): Linear(-1, 256, bias=True)
      (jobs): Linear(-1, 256, bias=True)
      (organizations): Linear(-1, 256, bias=True)
    )
    (convs): ModuleList(
      (0-1): 2 x HGTConv(-1, 256, heads=8)
    )
    (lin): Linear(256, 256, bias=True)
  )
)

In [ ]:
from tqdm.auto import tqdm
from datetime import datetime
batch_size = 32
num_node_types = len(train_data.node_types)
print('num_node_types', num_node_types)
one_hop_neighbors = (20 * batch_size)//num_node_types # per relationship type
two_hop_neighbors = (20 * 8 * batch_size)//num_node_types # per relationship type
three_hop_neighbors = (20 * 8 * 3 * batch_size)//num_node_types # per relationship type
num_neighbors = [one_hop_neighbors, two_hop_neighbors] # three_hop_neighbors
# num_neighbors [36, 363, 1454]

print('num_neighbors', num_neighbors)
print('avg_num_neighbors', [num_neighbors[0]/batch_size,num_neighbors[1]/batch_size,  num_neighbors[2]/batch_size if len(num_neighbors)==3 else 0 ])

train_sampler = equal_edgeweight_hgt_sampler(train_data, batch_size, True, 'triplet', 1, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)
val_sampler = equal_edgeweight_hgt_sampler(val_data, batch_size, True, 'triplet', 1, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)


learning_rate = 2e-4
# torch get optimizer by string name
optimizer = 'Adam'
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #2e-15


# create a tensorboard writer
from torch.utils.tensorboard import SummaryWriter
neighbors = '_'.join([str(n) for n in num_neighbors])


timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

writer = SummaryWriter(ROOT_FOLDER+f'runs/learningall_hgt_{timestamp}_margin05_pnorm{pnorm}_lr{learning_rate}_bs{batch_size}_neighbors_{neighbors}_head_{head}_hiddenchannels_{hidden_channels}_outchannels_{out_channels}_numheads_{num_heads}_numlayers_{num_layers}')
print('writer',ROOT_FOLDER+f'runs/learningall_hgt_{timestamp}_margin05_pnorm{pnorm}_llr{learning_rate}_bs{batch_size}_neighbors_{neighbors}_head_{head}_hiddenchannels_{hidden_channels}_outchannels_{out_channels}_numheads_{num_heads}_numlayers_{num_layers}')

model.train()
start_epoch = 1
total_minibatches = get_minibatch_count(train_data, batch_size)
for epoch in range(start_epoch, start_epoch+1000):
    for i, (same_nodetype, target_edge_type, minibatch) in tqdm(enumerate(train_sampler), total=total_minibatches):

        try:
            optimizer.zero_grad()
            # batching is different depending on if node types in edge are same or different
            print(target_edge_type)
            if same_nodetype:

                minibatch, edge_label_index, edge_label, input_edge_ids, global_node_ids = minibatch
                #print(minibatch['jobs'].x.device, edge_label_index.device, edge_label.device)
                loss = model(minibatch.to(device), target_edge_type, edge_label_index.to(device), edge_label.to(device))
                #loss, pos, neg = model(minibatch, target_edge_type, edge_label_index, edge_label)
            else:
                try:
                    minibatchpart1, minibatchpart2, edge_label_index, edge_label, input_edge_id, global_src_ids, global_dst_ids = minibatch
                except ValueError as err:
                    print('value error', err)
                    continue # for skill qual edges sometimes for some reason only 5 instead of 7 elements returned
                #print(minibatchpart1['jobs'].device, minibatchpart2['jobs'].device, edge_label_index.device, edge_label.device)
                loss = model(minibatchpart1.to(device), target_edge_type, edge_label_index.to(device), edge_label.to(device), minibatchpart2.to(device))


            loss.backward()
            optimizer.step()

            total_samples_seen = i * batch_size
            writer.add_scalar('Loss/train', loss.item(), total_samples_seen)

            if i == total_minibatches-1:
                print(f'{i} loss: {loss.item():.4f}')
                writer.add_scalar('Epoch Loss/train', loss.item(), total_samples_seen)

            # print loss and minibatch in the same line
            print(f'{i} loss: {loss.item():.4f}', end='\r')

            if i % 300 == 0 or i == total_minibatches-1:
                model.eval()
                with torch.no_grad():
                    val_loss = 0
                    for _ in range(3):
                        try:
                            same_nodetype, target_edge_type, minibatch = next(val_sampler)
                        except StopIteration:
                            val_sampler = iter(val_sampler)
                            same_nodetype, target_edge_type, minibatch = next(val_sampler)

                        if same_nodetype:
                            minibatch, edge_label_index, edge_label, input_edge_ids, global_node_ids = minibatch
                            #print(minibatch['jobs'].x.device, edge_label_index.device, edge_label.device)
                            val_loss += model(minibatch.to(device), target_edge_type, edge_label_index.to(device), edge_label.to(device))
                            #loss, pos, neg = model(minibatch, target_edge_type, edge_label_index, edge_label)
                        else:
                            try:
                                minibatchpart1, minibatchpart2, edge_label_index, edge_label, input_edge_id, global_src_ids, global_dst_ids = minibatch
                            except ValueError:
                                continue

                            #print(minibatchpart1['jobs'].device, minibatchpart2['jobs'].device, edge_label_index.device, edge_label.device)
                            val_loss += model(minibatchpart1.to(device), target_edge_type, edge_label_index.to(device), edge_label.to(device), minibatchpart2.to(device))

                val_loss /= 3
                if i == 0:
                    writer.add_scalar('Epoch Loss/val', val_loss, total_samples_seen)
                    writer.add_scalar('Loss/val', val_loss, total_samples_seen)
                elif i == total_minibatches-1:
                    writer.add_scalar('Epoch Loss/val', val_loss, total_samples_seen)
                else:
                    writer.add_scalar('Loss/val', val_loss, total_samples_seen)


                print(f'val_loss: {val_loss:.4f}', end='\r')
                model.train()

            writer.flush()

            if i % 1000 == 0 or i == total_minibatches-1:
                folder = 'models'
                if not os.path.exists(folder):
                    os.makedirs(folder)

                run_folder = ROOT_FOLDER+f'{folder}/learningall_hgt_{timestamp}_margin05_pnorm{pnorm}_llr{learning_rate}_bs{batch_size}_neighbors_{neighbors}_head_{head}_hiddenchannels_{hidden_channels}_outchannels_{out_channels}_numheads_{num_heads}_numlayers_{num_layers}'
                if not os.path.exists(run_folder):
                    os.makedirs(run_folder)

                print('saving model to', run_folder)
                # save model and optimizer
                is_epoch = f'Ep{epoch}_' if i == total_minibatches-1 else ''
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    }, run_folder+f'/{is_epoch}model_samplesseen{total_samples_seen}.pt')

        except IndexError:
            print('indexerror')
            pass

writer.close()